In [ ]:
import pandas as pd
gpt_tag=pd.read_csv("/content/drive/MyDrive/colab_github/sooljari-crawling/gpt/gpt_tag_weight.csv")


In [ ]:
gpt_tag['tags']

0      ['와인', '복숭아', '디저트', '당도', '발효', '맛', '향', '냉동...
1      ['와인', '화이트와인', '과일향', '아로마틱', '달콤함', '산미', '고...
2      ['매실주', '황매실', '100%', '제주도꿀', '전통제조방법', '향미',...
3      ['메밀향', '스파이시한', '반전매력', '프레시', '화사한', '정통', '...
4                          ['삼양주', '탁주', '과일향', '새콤달콤함']
                             ...                        
143    ['부드럽고깨끗한맛', '매실의풍미', '달콤함', '12도도수', '매화꽃디자인'...
144    ['매취순', '매실주', '청매실', '프리미엄', '국산', '해남', '보해매...
145    ['대윤가야곡주조', '가야곡왕주', '약주', '궁중술', '왕주', '역사', ...
146    ['대표막걸리', '인공감미료없음', '첨가물없음', '가야양조장', '발효기술',...
147    ['발효', '오랜숙성', '저온숙성', '기다림의미학', '풍부한맛', '풍부한향...
Name: tags, Length: 148, dtype: object

In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import os
import openai
import json
import csv
import pandas as pd

f = open("chatgptkey.txt", 'r')
line = f.readline()
openai.api_key = line

from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key
)

df1 = pd.read_csv('drinkdata.csv')
frame=pd.DataFrame(df1)


In [ ]:
rownum=[57]

In [ ]:
df1.iloc[rownum]

In [ ]:
output=["","","",""]
import re
i=0
for row in df1.iloc[rownum]['텍스트 내용']:
  description =  row # 원하는 설명문을 입력하세요
  response = client.chat.completions.create(
  temperature=0.2,
  model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": "주류 데이터를 검색하기 위한 태그를 설명문으로부터 추출하고 싶어. 술에 대한 정보를 포함하는 10자 이내의 태그와 각 태그의 중요도를 ,로 묶어서 출력하고 싶어. "},
    {"role": "system", "content": "또한 추출한 태그의 중요도를 1에서 10 사이의 수치로 표현해줘. 각 태그가 주관적인지 객관적인지 판단해서, 맛이나 향 등을 포함하거나 주관적이라면 높은 점수를 줘."},
    {"role": "user", "content": description},
  ]
  )
  tempstr=response.choices[0].message.content
  print(tempstr)
  if '태그: ' in tempstr and '중요도: ' in tempstr: #fine string
    # add tags
    lines=tempstr.split('\n')
    if "" in lines:
      lines.remove("")
    if len(lines)!=1 and '태그: ' in lines[0] and '중요도: ' in lines[1]:
      #good
      print("good")
      output[i]=lines[0].replace("태그:","").replace(" ","")+"\n"+lines[1].replace("중요도:","").replace(" ","")

  elif '태그: ' in tempstr and len(tempstr.split('\n'))==1:
    print("oneline")
    p = re.compile('\(([^)]+)')
    tempstr=tempstr+"\n"
    for toint in p.findall(tempstr):
      tempstr=tempstr+toint+","
    tempstr=tempstr+"\n"
    tempstr=tempstr.replace(",\n","")
    tempstr=re.sub('\([^)]+\)','',tempstr)
    print(tempstr)
    output[i]=tempstr.replace("태그:","").replace(" ","")
  i=i+1


장밋빛의 로제 스파클링 와인, 케이크, 스위트 와인, 디저트


In [ ]:
finaloutput

In [ ]:
rownum=[39, 57, 88, 130]

"['옥주프리미엄', '곡물향', '원기옥']"

In [ ]:
finaloutput
for e in finaloutput:
  finaloutput[finaloutput.index(e)]=e.replace(" ","")

In [ ]:
#숫자만 존재하면 숫자만 리턴하고, 아니라면 문자열 파싱함

import re
finaltags=[]
finalweight=[]

for temp in finaloutput:
  line=temp.split("\n")
  tags=line[0].split(",")
  finaltags.append(tags)
  flag=True
  try:
    for s in line[1].split(","):
      if not s.isdigit():
        flag=False
        break
    if not flag:
      p = re.compile('\(([^)]+)')
      out=[]
      for toint in p.findall(line[1]):
        out.append(int(toint))
      print(out)
      finalweight.append(out)
    else:
      out=[]
      for toint in line[1].split(','):
        out.append(int(toint))
      print(out)
      finalweight.append(out)
  except:
    finalweight.append([])


[8, 6, 7, 7, 9, 10, 7, 6, 5, 4, 6, 8]
[6, 5, 5, 8, 7, 6]
[10, 9, 7, 8, 6]
[7, 8, 9]


In [ ]:
rownum

[39, 57, 88, 130]

In [ ]:
cnt=0
errlst = []
for item in gpt_tag['tags']:
  if(type(item)==type("str")):
    newstr=item.replace("#","").replace("\n","").replace(".","")
    if(newstr[0]!='#'):
      print(cnt)
      errlst.append(cnt)
  else:
    print(cnt)
    errlst.append(cnt)
  cnt=cnt+1

In [ ]:
for idx,row in gpt_tag.iterrows():
  #print(row['tags'])
  if("#" in row['tags']):
    print(idx)
    print(row['tags'])

128
['#옥수수', '#막걸리', '#진짜']
129
['#요구르트', '#자두풍미', '#새콤']
131
['#부드러운', '#참외풍미', '#달콤']
141
['#막걸리#신선함#풍부한맛#향기로움#효모의맛#시간의변화#프리미엄#수제#부드러움#다양한맛']


In [ ]:
gpt_tag.loc[141,'tags']

"['막걸리','신선함','풍부한맛','향기로움','효모의맛','시간의변화','프리미엄','수제','부드러움','다양한맛']"

In [ ]:
#print(str(errlst[0])+" "+gpt_tag['텍스트 내용'][errlst[0]]+".")

In [ ]:
gpt_tag.to_csv('gpt_tag_weight_refined.csv')